# Import Libraries

In [ ]:
import pandas as pd
import os
import os.path
from zipfile import ZipFile
import string
import pandas as pd

# Establish wd and test concepts

In [ ]:
os.getcwd() #Correct folder
#os.path.isdir(os.listdir()[1])

In [ ]:
#Walk through all folders and subfolders to populate a list, then unpack the nested list
for item in os.listdir():
    if os.path.isdir(item):
        nested_file_list = [sublist[2] for sublist in os.walk(item)]
        flattened_file_list = [x for sublist in nested_file_list for x in sublist]

# Program v1 : Identifies length issue and bad zip file

In [ ]:
#Initiate Empty Dictionary
problem_files = {}
#Try a for loop to join the two lists 
for item in os.listdir():
    if os.path.isdir(item):
        root_list = [sublist[0] for sublist in os.walk(item)]
        fname_list = [sublist[2] for sublist in os.walk(item)]
        for i in range(0, len(root_list)): #Loop through all root and subroot folders
            if len(fname_list[i]) > 0: #Check if there are files in the folder
                for k in range(0, len(fname_list[i])): #Loop through the files contained in the subfolder
                    full_path = root_list[i]+ "\\" + fname_list[i][k] #Concatenate root and file name
                    if full_path[-3:] == 'zip':
                        try: 
                            zip_file_names = ZipFile(full_path, mode = 'r').namelist() #List of underlying zip folders
                        except:
                            problem_files[root_list[0]] = 'Bad Zip File'
                        else:    
                            for j in range(0, len(zip_file_names)): #Loop through underlying zip folders 
                                full_path = root_list[i]+ "\\" + fname_list[i][k]+ "\\" + zip_file_names[j]
                         
                    if ((len(full_path + os.getcwd())-96)  > 240) and (root_list[0] not in problem_files) :
                        problem_files[root_list[0]] = f'Name too long {len(full_path+os.getcwd())} '
                    
                
    

In [ ]:
problem_files

# Program v2: Identifies which files are problematic and checks for special characters

In [ ]:
#Initiate Empty Dictionary
problem_files = {}

#File length reduction factor to account for moving files to other directories
length_reduct = 90

#Define special characters 
special_chars = '!"#$%&\'()*+,:;<=>?@[\\]^`{|}~'
non_special_chars = '._-/'

#Main Program 
for item in os.listdir():
    if os.path.isdir(item):
        root_list = [sublist[0] for sublist in os.walk(item)]
        fname_list = [sublist[2] for sublist in os.walk(item)]
        len_issue_list = [] 
        char_issue_list = []
        other_issue_list = []
        issue_dict = {'len_issues':'', 'char_issues':'', 'other_issues':''}
        
        for i in range(0, len(root_list)): #Loop through all root and subroot folders
            if len(fname_list[i]) > 0: #Check if there are files in the folder
                for k in range(0, len(fname_list[i])): #Loop through the files contained in the subfolder
                    full_path = root_list[i]+ "\\" + fname_list[i][k] #Concatenate root and file name
                    if full_path[-3:] == 'zip':
                        try: 
                            zip_file_names = ZipFile(full_path, mode = 'r').namelist() #List of underlying zip folders
                        except:
                            other_issue_list.append(f'Bad Zip File{fname_list[i]}') #Appends bad zip issue to list
                        else:    
                            for j in range(0, len(zip_file_names)): #Loop through underlying zip folders 
                                full_path = root_list[i]+ "\\" + fname_list[i][k]+ "\\" + zip_file_names[j] #Spell out full path  
                                if any(char in special_chars for char in zip_file_names[j]): #Check for special chars in zip file names
                                    char_issue_list.append(f'Special Character in zip file_name = {zip_file_names[j]}')
                    #Check length issue
                    if ((len(full_path + os.getcwd())-length_reduct)  > 240): #Check path length against zip constraint
                        len_issue_list.append(f'Name too long {len(full_path+os.getcwd())} file_name = {fname_list[i][k]}') #Append length issue to list     
                    #Check for special characters 
                    if any(char in special_chars for char in fname_list[i][k]): #Check for special chars in zip file names
                                    char_issue_list.append(f'Special Character in file_name = {fname_list[i][k]}') 
        if sum(map(len, [len_issue_list, char_issue_list, other_issue_list]))  > 0   : #Append k,v pair to dict if there exists issues
            issue_dict['len_issues'], issue_dict['char_issues'],issue_dict['other_issues'] = len_issue_list, char_issue_list, other_issue_list
            problem_files[root_list[0]] = issue_dict
                    
#Show output
problem_files

# Convert to df and export

In [ ]:
temp = pd.DataFrame(problem_files.values(), index = problem_files.keys()).reset_index()
#temp.melt(id_vars=['index'])
temp.to_csv('Data_issues.csv')